In [ ]:
# Find data set
# price, social media following, home/away = win/lose
# scanning for games on smarkets, scraping social media followings then predicting

In [ ]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib
import googlesearch
from fuzzywuzzy import fuzz
import json
import os
import re
import sys

from bs4 import BeautifulSoup
from contextlib import closing
from selenium.webdriver import Firefox # pip install selenium
from selenium.webdriver import Chrome # pip install selenium
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

list_ = ["https://twitter.com/GoalProfits/lists/english-championship/members",
        "https://twitter.com/premierleague/lists/premier-league-clubs/members?lang=en",
        "https://twitter.com/GoalProfits/lists/german-bundesliga",
        "https://twitter.com/GoalProfits/lists/german-2-bundesliga",
        "https://twitter.com/GoalProfits/lists/brazilian-serie-a/members",
        "https://twitter.com/GoalProfits/lists/french-ligue-1/members",
        "https://twitter.com/GoalProfits/lists/spanish-primera/members",
        "https://twitter.com/GoalProfits/lists/italian-serie-a/members",
        "https://twitter.com/GoalProfits/lists/scottish-premiership/members",
        "https://twitter.com/GoalProfits/lists/dutch-eredivisie/members",
        "https://twitter.com/GoalProfits/lists/north-american-mls/members",
        "https://twitter.com/GoalProfits/lists/norwegian-tippeligaen/members",
        "https://twitter.com/GoalProfits/lists/swedish-allsvenskan/members",
        "https://twitter.com/GoalProfits/lists/irish-premier/members"]  # can increase data set, but I am lazy so these are all


csv_list = ['E1 (1).csv', 'E0 (1).csv', 'E1.csv', 'N1 (1).csv', 'F1 (1).csv', 'F1 (2).csv', 'N1 (2).csv',
            'N1.csv', 'F1.csv', 'SP1.csv', 'SP1 (1).csv', 'SP1 (2).csv', 'SC0 (2).csv', 'SC0.csv',
            'SC0 (1).csv', 'D1 (2).csv', 'D2 (2).csv', 'D2 (1).csv', 'D1 (1).csv', 'D2.csv', 'E1 (2).csv',
            'D1.csv', 'E0 (2).csv']
# from http://www.football-data.co.uk/data.php  I didn't automate this as this is not a good data source
# there are a lot of stats missing and if you looked you can probably find better sources

In [77]:
def fuzz_comp(string):
    best_score = 0
    for i in twitter_stats:
        if (i in string) or (string in i):
            return(twitter_stats[i])
        similarity = fuzz.token_sort_ratio(string, i)
        if similarity > best_score:
            best_score = similarity
            best_ = twitter_stats[i]
            best = i
        if best_score < 0.6:
            return 0
    return best_

def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')



def twitter_data(lists_):
    handles = {}
    for twitter_page in list_:
        result = requests.get(twitter_page)
        src = result.content
        soup = BeautifulSoup(src, 'html.parser')
        members = soup.find_all('div',{'class':"activity-user-profile-content"})
        for i in members:
            name = i.find(class_ = "fullname").text
            handle = i.find(class_= "username u-dir u-textTruncate").text
            handles[deEmojify(name)] = handle

    twitter_stats = handles.copy()
    for handle in handles.keys():
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
        url = "https://socialblade.com/twitter/user/" + handles[handle][1:]
        src = requests.get(url, headers=headers).text

        soup = BeautifulSoup(src, 'html.parser')
        atv = soup.find('div',{'class':"YouTubeUserTopInfo"})
        followers = atv.text.split()[1].replace(',', '')
        try:
            stats = soup.findAll("p",{"style" : "font-size: 1.6em; color: #41a200; padding-top: 10px; font-weight: 600; margin-top: -15px;"})
            retweets = stats[1].text.replace(',', '')
            likes = stats[2].text.replace(',', '')
        except:
            likes = tweets = 0
        print(handle + ":", followers, retweets, likes)
        if 'Football Club' in handle:
            twitter_stats[handle.split()[0]] = [followers, retweets, likes]
            del twitter_stats[handle]
        else:
            twitter_stats[handle] = [followers, retweets, likes]
    return twitter_stats

def football_data(csv_list):
    # df = pd.read_csv("http://www.football-data.co.uk/mmz4281/1617/E0.csv") and download all directly from website
    df = pd.read_csv('E0.csv')
    for i in csv_list:
        try:
            df.append(pd.read_csv(i))
        except:
            continue
    return df


def cleaning_data(df, twitter_stats):
    df['statsh'] = [fuzz_comp(w) for w in df.HomeTeam]
    df = df[df.statsh != 0]
    df['statsa'] = [fuzz_comp(w) for w in df.AwayTeam]
    df = df[df.statsa != 0]

    df['stat1'] = [w[0] for w in df.statsh]
    df['stat2'] = [w[1] for w in df.statsh]
    df['stat3'] = [w[2] for w in df.statsh]

    df['stat_1'] = [w[0] for w in df.statsa]
    df['stat_2'] = [w[1] for w in df.statsa]
    df['stat_3'] = [w[2] for w in df.statsa]

    df1 = df[['FTR', 'HomeTeam', 'stat1', 'stat2', 'stat3', 'Div', 'B365H', 'Date', 'AwayTeam', 'stat_1', 'stat_2', 'stat_3']].copy()
    df1.rename(columns={'FTR': 'result', 'HomeTeam': 'team', 'B365H': 'odds', 'Date': 'date', 'AwayTeam': 'opo',
                        'stat1': 'h1', 'stat2': 'h2', 'stat3': 'h3',
                        'stat_1': 'a1', 'stat_2': 'a2', 'stat_3': 'a3'}, inplace=True)
    df1['location'] = 0
    df1['result'] = [int(w) for w in df1['result'] == 'H']
    df2 = df[['FTR', 'AwayTeam', 'stat_1', 'stat_2', 'stat_3', 'Div', 'B365A', 'Date', 'HomeTeam', 'stat1', 'stat2', 'stat3']].copy()
    df2['location'] = 1
    df2.rename(columns={'FTR': 'result', 'AwayTeam': 'team', 'B365A': 'odds', 'Date': 'date', 'HomeTeam': 'opo',
                        'stat_1': 'h1', 'stat_2': 'h2', 'stat_3': 'h3',
                        'stat1': 'a1', 'stat2': 'a2', 'stat3': 'a3'}, inplace=True)
    df2['result'] = [int(w) for w in df2['result'] == 'A']
    dataset = pd.concat([df1, df2])
    dataset['odds2'] = 1 / dataset['odds']
    dataset['date'] = [2019 - int(w.split("/")[2]) for w in dataset.date]
    dataset.reset_index(inplace=True)
    dataset.drop(['index','team','opo'], axis=1, inplace=True)
    return dataset

def machine_learning():
        # havent done yet, I couldn't find a suitable model
        # am thinking that a deep reinforcement learning would work best
        # as you want the result to be the odds multiplied by whether it was a win
        # THe lack of variables of this model means that even if I were to get a good model, there would be too much uncertainty
        # too much chaos, so I would need to add other variables such as which players are playing, which would require a larger dataset and a much better gpu
        # adding win streak, mean wins that season would also help
        # converting years to season would be good, but difficult given that different leagues have different season dates
        # adding goal keeper and defence statistics would mean you could analyse the possibility of a draw
        # adding position in league and how far they are from te player above and below
        # a sentiment analysis of the tweets that the teams are being tagged in might also be interesting and fun to make
            # you could even use a sentiment analysis of player tweets to see how potitive a player is  and how distracted they are
        return model
    
def scan_smarkets(): # could scan oddschecker to get best odds, but qould require man many betting accounts
        # plus smarkets tends to have best odds along with betfair (except smarkets takes only 2% comission compared to 5%)
        # I have to use selenium as have had some forbidden access errors with beautifulsoup 
    

In [130]:
df1 = df[['FTR', 'HomeTeam', 'Div', 'B365H', 'Date', 'AwayTeam']].copy()
df1.rename(columns={'FTR': 'result','HomeTeam': 'team', 'B365H': 'odds', 'Date': 'date', 'AwayTeam': 'opo'}, inplace=True)
df1['location'] = 0
df1.result = [int(w) for w in df1['result'] == 'H']
df2 = df[['FTR', 'AwayTeam', 'Div', 'B365A', 'Date', 'HomeTeam']].copy()
df2['location'] = 1
df2.rename(columns={'FTR': 'result', 'AwayTeam': 'team', 'B365A': 'odds', 'Date': 'date', 'HomeTeam': 'opo'}, inplace=True)
df2.result = [int(w) for w in df2['result'] == 'A']
dataset = pd.concat([df1, df2])
dataset.odds = 1 / dataset.odds
dataset.date = [2019 - int(w.split("/")[2]) for w in dataset.date]

In [189]:
fuzz.token_sort_ratio('Brighton', 'Brighton & Hove Albion')

57

In [77]:


df['statsh'] = [fuzz_comp(w) for w in df.HomeTeam]
df = df[df.statsh != 0]
df['statsa'] = [fuzz_comp(w) for w in df.AwayTeam]
df = df[df.statsa != 0]

df['stat1'] = [w[0] for w in df.statsh]
df['stat2'] = [w[1] for w in df.statsh]
df['stat3'] = [w[2] for w in df.statsh]

df['stat_1'] = [w[0] for w in df.statsa]
df['stat_2'] = [w[1] for w in df.statsa]
df['stat_3'] = [w[2] for w in df.statsa]

df1 = df[['FTR', 'HomeTeam', 'stat1', 'stat2', 'stat3', 'Div', 'B365H', 'Date', 'AwayTeam', 'stat_1', 'stat_2', 'stat_3']].copy()
df1.rename(columns={'FTR': 'result', 'HomeTeam': 'team', 'B365H': 'odds', 'Date': 'date', 'AwayTeam': 'opo',
                    'stat1': 'h1', 'stat2': 'h2', 'stat3': 'h3',
                    'stat_1': 'a1', 'stat_2': 'a2', 'stat_3': 'a3'}, inplace=True)
df1['location'] = 0
df1['result'] = [int(w) for w in df1['result'] == 'H']
df2 = df[['FTR', 'AwayTeam', 'stat_1', 'stat_2', 'stat_3', 'Div', 'B365A', 'Date', 'HomeTeam', 'stat1', 'stat2', 'stat3']].copy()
df2['location'] = 1
df2.rename(columns={'FTR': 'result', 'AwayTeam': 'team', 'B365A': 'odds', 'Date': 'date', 'HomeTeam': 'opo',
                    'stat_1': 'h1', 'stat_2': 'h2', 'stat_3': 'h3',
                    'stat1': 'a1', 'stat2': 'a2', 'stat3': 'a3'}, inplace=True)
df2['result'] = [int(w) for w in df2['result'] == 'A']
dataset = pd.concat([df1, df2])
dataset['odds2'] = 1 / dataset['odds']
dataset['date'] = [2019 - int(w.split("/")[2]) for w in dataset.date]
dataset.reset_index(inplace=True)
dataset.drop(['index','team','opo'], axis=1, inplace=True)

In [75]:
csv_list = ['E1 (1).csv', 'E0 (1).csv', 'E1.csv', 'N1 (1).csv', 'F1 (1).csv', 'F1 (2).csv', 'N1 (2).csv',
            'N1.csv', 'F1.csv', 'SP1.csv', 'SP1 (1).csv', 'SP1 (2).csv', 'SC0 (2).csv', 'SC0.csv',
            'SC0 (1).csv', 'D1 (2).csv', 'D2 (2).csv', 'D2 (1).csv', 'D1 (1).csv', 'D2.csv', 'E1 (2).csv',
            'D1.csv', 'E0 (2).csv']


df = pd.read_csv('E0.csv') #for an earlier version of Excel, you may need to use the file extension of 'xls'
for i in csv_list:
    try:
        df = df.append(pd.read_csv(i))
    except:
        continue
df['statsh'] = [fuzz_comp(w) for w in df.HomeTeam]

In [95]:
dataset

,result,h1,h2,h3,Div,odds,date,a1,a2,a3,location,odds2
0,1,12797570,1142,5945,E0,0.877193,0,672400,16,119,0,1.14
1,0,1561401,73,511,E0,0.083333,0,7350439,247,1314,0,12.00
2,0,476661,16,95,E0,0.512821,0,267620,12,61,0,1.95
3,1,441223,13,72,E0,0.381679,0,1136501,18,117,0,2.62
4,0,889714,11,66,E0,0.333333,0,1940696,138,1029,0,3.00
5,0,636798,12,72,E0,0.526316,0,250978,10,44,0,1.90
6,1,3817175,195,1214,E0,0.769231,0,1288550,61,345,0,1.30
7,0,1418790,29,179,E0,0.454545,0,492655,90,651,0,2.20
8,0,1449134,27,125,E0,0.222222,0,15121561,624,3238,0,4.50
9,1,20457462,1164,5272,E0,0.476190,0,13504561,645,3228,0,2.10


In [23]:
df = pd.DataFrame({'a':[0,0,1,1,2,9,1], 'b':[0,'hello',0,1,7,7,8], 'c':[0,1,0,1,13,13,221], 'd':[0,3,3,3,1,0,1]})

In [175]:
twitter_stats = handles.copy()
for handle in handles.keys():
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
    url = "https://socialblade.com/twitter/user/" + handles[handle][1:]
    src = requests.get(url, headers=headers).text
    
    soup = BeautifulSoup(src, 'html.parser')
    atv = soup.find('div',{'class':"YouTubeUserTopInfo"})
    followers = atv.text.split()[1].replace(',', '')
    try:
        stats = soup.findAll("p",{"style" : "font-size: 1.6em; color: #41a200; padding-top: 10px; font-weight: 600; margin-top: -15px;"})
        retweets = stats[1].text.replace(',', '')
        likes = stats[2].text.replace(',', '')
    except: likes = tweets = 0
    # print(handle + ":", followers, retweets, likes)
    if 'Football Club' in handle:
        twitter_stats[handle.split()[0]] = [followers, retweets, likes]
    else:
        twitter_stats[handle] = [followers, retweets, likes]

Fulham Football Club
Watford Football Club
Torino Football Club
Rangers Football Club
Celtic Football Club


In [178]:
from fuzzywuzzy import fuzz
def fuzz_comp(string):
    best_score = 0
    for i in twitter_stats:
        similarity = fuzz.token_sort_ratio(string, i)
        if similarity > best_score:
            best_score = similarity
            best_ = twitter_stats[i]
            best = i
    print(string, best)
    return best_
dataset['stats'] = [fuzz_comp(w) for w in dataset.team]

In [179]:
dataset['stats'] = [fuzz_comp(w) for w in dataset.team]

Liverpool Liverpool FC
West Ham West Ham United
Bournemouth AFC Bournemouth
Burnley Burnley FC
Crystal Palace Crystal Palace F.C.
Watford Watford
Tottenham Tottenham Hotspur
Leicester Leicester City
Newcastle Newcastle United FC
Man United Manchester United
Arsenal Arsenal
Aston Villa Aston Villa
Brighton Brighton & Hove Albion 
Everton Everton
Norwich Norwich City FC
Southampton Southampton FC
Man City Manchester City
Sheffield United Sheffield United
Chelsea Chelsea FC
Wolves Wolves
Aston Villa Aston Villa
Norwich Norwich City FC
Brighton Brighton & Hove Albion 
Man United Manchester United
Sheffield United Sheffield United
Watford Watford
Liverpool Liverpool FC
Bournemouth AFC Bournemouth
Tottenham Tottenham Hotspur
Wolves Wolves
Southampton Southampton FC
Chelsea Chelsea FC
Crystal Palace Crystal Palace F.C.
Leicester Leicester City
Man City Manchester City
Newcastle Newcastle United FC
West Ham West Ham United
Burnley Burnley FC
Everton Everton
Arsenal Arsenal
Liverpool Liverpool 

In [172]:
fuzz.ratio('Watford', 'Waterford FC')

74

In [75]:
stats = soup.findAll("p",{"style" : "font-size: 1.6em; color: #41a200; padding-top: 10px; font-weight: 600; margin-top: -15px;"})
stats[1].text

'54'

In [39]:
result = requests.get("https://twitter.com/GoalProfits/lists/english-championship/members")
src = result.content
soup = BeautifulSoup(src, 'html.parser')
atv = soup.find('div',{'class':"YouTubeUserTopInfo"})
for e in atv:
    print(e)
    break

<div class="activity-user-profile-content">
<div class="content">
<div class="stream-item-header">
<a class="account-group js-user-profile-link" href="/LUFC" rel="noopener">
<img alt="" class="avatar js-action-profile-avatar" data-user-id="1024095475" src="https://pbs.twimg.com/profile_images/1160984924222119937/qK_NijGO_normal.png">
<strong class="fullname">Leeds United</strong><span class="UserBadges"><span class="Icon Icon--verified"><span class="u-hiddenVisually">Verified account</span></span></span><span class="UserNameBreak"> </span><span class="username u-dir u-textTruncate" dir="ltr">@<b>LUFC</b></span>
</img></a>
</div>
<p class="bio u-dir" dir="ltr">
                The official Twitter account of Leeds United <a class="twitter-hashtag pretty-link js-nav" data-query-source="hashtag_click" dir="ltr" href="/hashtag/LUFC?src=hash"><s>#</s><b>LUFC</b></a>
</p>
</div>
</div>


In [42]:
e.find(class_ = "fullname").text
e.find(class_= "username u-dir u-textTruncate").text

'Leeds United'

In [44]:
e.find(class_= "username u-dir u-textTruncate").text

'@LUFC'

In [55]:
handles['Leeds United'][1:]

'LUFC'

In [93]:
for i, e in enumerate([5,4,3,2,100]):
    print(i, e)

0 5
1 4
2 3
3 2
4 100


In [149]:
from fuzzywuzzy import fuzz
def fuzz_comp(string):
    best_score = 0
    for i in twitter_stats:
        similarity = fuzz.ratio(string, i)
        if similarity > best_score:
            best_score = similarity
            best_ = twitter_stats[i]
    return best_

In [146]:
result = requests.get("https://smarkets.com/listing/sport/football?period=today")
src = result.content
soup = BeautifulSoup(src, 'xml')
atv = soup.find_all('div',{'class':"contract open"})
for e in atv:
    arb = e.find(class_ = "name")
    name_ = arb.find(class_ = "formatted-content")
    price = e.find(class_="price tick buy   formatted-price numeric-value")
    price2 = e.find(class_="price tick sell   formatted-price numeric-value")
    if price == None: p = "No liquidity"
    else: p = price.string
    if price2 == None: p2 = "No liquidity"
    else: p2 = price2.string
    print(name_.string, "  buy:", p, "  sell:", p2)
    print()
result = requests.get("https://smarkets.com/event/886716/politics/uk/uk-party-leaders/next-conservative-leader")

{'Leeds United': ['447055', '54', '342'],
 'Swansea City AFC': ['1010885', '8', '63'],
 'Middlesbrough FC': ['317772', '13', '97'],
 'West Bromwich Albion': ['986001', '31', '197'],
 'Millwall FC': ['114339', '5', '28'],
 'Derby County': ['293905', '29', '155'],
 'Reading FC': ['332879', '9', '71'],
 'Huddersfield Town': ['198163', '8', '32'],
 'Barnsley FC': ['1075', '8', 0],
 'Hull City': ['441691', '11', '67'],
 'Bristol City FC': ['174541', '7', '44'],
 'Brentford FC': ['128400', '4', '29'],
 'Queens Park Rangers FC': ['425993', '12', '71'],
 'Charlton Athletic FC': ['156755', '7', '50'],
 'Birmingham City FC': ['315474', '13', '77'],
 'Stoke City FC': ['1055818', '10', '49'],
 'Cardiff City FC': ['327737', '17', '59'],
 'Nottingham Forest FC': ['325856', '28', '173'],
 'Luton Town FC': ['90777', '6', '39'],
 'Fulham Football Club': ['506214', '22', '160'],
 'Manchester United': ['20451886', '1164', '5272'],
 'Burnley FC': ['441014', '13', '72'],
 'Watford Football Club': ['636681'

In [74]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
result = requests.get('https://www.oddschecker.com/football',headers=headers)
src = result.text
soup = BeautifulSoup(src, 'html.parser')
atv = soup.find_all('div',{'class':"contract-items  open"})
for e in atv:
    print(e)
    break

In [15]:
atv = soup.find_all('div',{'class':"contract-items null"})
for e in atv:
    print(e)
    break

<div class="contract-items null"><span class="contract-item"><span class="contract-label"><span class="contract-name"></span></span></span><span class="contract-item"><span class="contract-label"><span class="contract-name"></span></span></span><span class="contract-item"><span class="contract-label"><span class="contract-name"></span></span></span></div>


In [75]:

soup

<!DOCTYPE doctype HTML>
<html lang="en"><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><link href="https://static.oddschecker.com/" rel="dns-prefetch"/><link crossorigin="true" href="https://static.oddschecker.com/" rel="preconnect"/><link as="font" crossorigin="true" href="https://static.oddschecker.com/OC/fonts/NunitoRegular/nunito-sans-v3-latin-regular.woff2" rel="preload" type="font/woff2"/><link as="font" crossorigin="true" href="https://static.oddschecker.com/OC/fonts/NunitoSemiBold/nunito-sans-v3-latin-600.woff2" rel="preload" type="font/woff2"/><link as="font" crossorigin="true" href="https://static.oddschecker.com/OC/fonts/NunitoBold/nunito-sans-v3-latin-700.woff2" rel="preload" type="font/woff2"/><link as="font" crossorigin="true" href="https://static.oddschecker.com/OC/fonts/NunitoBlack/nunito-sans-v3-latin-900.woff2" rel="preload" type="font/woff2"/><style type="text/css">
      @font-face {
        font-family: 'Nunito Sans';
        font-style: 

In [65]:
url='https://www.bet365.com/#/AC/B1/C1/D13/E0/F2/J99/Q1/F%5E24/'
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html)

test = soup.find_all('span')
for span in test:
    print(span)

In [67]:
response = requests.get(url, headers=headers)

In [68]:
response.text

'\r\n\r\n<!DOCTYPE html><html lang="en">\r\n  <link rel="shortcut icon" type="image/x-icon" href="/favicons/main-favicon.ico">\r\n  <head>\r\n    <META http-equiv="Content-Type" content="text/html; charset=utf-16">\r\n    <meta name="google" content="notranslate">\r\n    <meta http-equiv="content-language" content="en"><!--197-0-1--><meta name="description" content="One of the world\'s leading online gambling companies. The most comprehensive In-Play service. Watch Live Sport. We stream over 140,000 events. Bet on Sports. Play Now on Casino, Poker, Games, Vegas and Bingo.">\r\n    <meta name="keywords" content="Sports Betting, Live In-Play, Live Streaming, Place Bet, Soccer Highlights, Premier League, Champions League, Casino, Poker, Games, Vegas, Bingo">\r\n    <meta http-equiv="X-UA-Compatible" content="requiresActiveX=true">\r\n    <title>bet365 - Sports Betting, Casino, Poker, Games, Vegas, Bingo</title>\r\n    <link rel="stylesheet" type="text/css" href="/home/style/sportsbook_vB2

In [ ]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib
#import googlesearch
from fuzzywuzzy import fuzz

In [ ]:


import json
import os
import re
import sys

from bs4 import BeautifulSoup
from contextlib import closing
from selenium.webdriver import Firefox # pip install selenium
from selenium.webdriver import Chrome # pip install selenium
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

def main():
    htmls = selenium()
    for html in htmls:
        parse(html)

def selenium():
    with closing(Firefox()) as browser:
        browser.get('https://www.betfair.com/sport/basketball')
        WebDriverWait(browser, timeout=10).until(EC.presence_of_element_located((By.XPATH, "//span[@class='section-header-label']")))
        element = browser.find_element_by_xpath("//*[contains(text(), 'NBA 2015-2016')]")
        element.click()
        WebDriverWait(browser, timeout=10).until(EC.presence_of_element_located((By.XPATH, "//span[@class='section-header-label']")))
        elementsA = browser.find_elements_by_xpath("//li[@class='com-coupon-line avb-row market-avb large ']")
        numOfElements = len(elementsA)

    htmls = []
    for i in range(0, numOfElements):
      element = browser.find_element_by_xpath("//*[contains(text(), 'NBA 2015-2016')]")
      element.click()
      WebDriverWait(browser, timeout=10).until(EC.presence_of_element_located((By.XPATH, "//span[@class='away-team-name']")))
      # elements = browser.find_elements_by_xpath("//li[@class='com-coupon-line avb-row market-avb large ']")
      elements = browser.find_elements_by_xpath("//div[@class='details-event']")
      print(i)
      elements[i].click()
      WebDriverWait(browser, timeout=10).until(EC.presence_of_element_located((By.XPATH, "//div[@class='ui-expandable ui-expandable-selected com-expandable-header-anchor has-deeplink']")))
      htmls.append(browser.page_source)
      browser.back()
      WebDriverWait(browser, timeout=10).until(EC.presence_of_element_located((By.XPATH, "//span[@class='section-header-label']")))
    return htmls 


#   numOfElements = getNumberOfElements()
#   # print(numOfElements)
#   htmls = []
#   for i in range(0, numOfElements):
#     htmls.append(getHtmlFromElementIndex(i))
#   return htmls

# def getNumberOfElements():
#   with closing(Firefox()) as browser:
#     browser.get('https://www.betfair.com/sport/basketball')
#     WebDriverWait(browser, timeout=10).until(EC.presence_of_element_located((By.XPATH, "//span[@class='section-header-label']")))
#     element = browser.find_element_by_xpath("//*[contains(text(), 'NBA 2015-2016')]")
#     element.click()
#     WebDriverWait(browser, timeout=10).until(EC.presence_of_element_located((By.XPATH, "//span[@class='section-header-label']")))
#     elements = browser.find_elements_by_xpath("//li[@class='com-coupon-line avb-row market-avb large ']")
#     return len(elements)

# def getHtmlFromElementIndex(index):
#   with closing(Firefox()) as browser:
#     browser.get('https://www.betfair.com/sport/basketball')
#     WebDriverWait(browser, timeout=10).until(EC.presence_of_element_located((By.XPATH, "//span[@class='section-header-label']")))
#     element = browser.find_element_by_xpath("//*[contains(text(), 'NBA 2015-2016')]")
#     element.click()
#     WebDriverWait(browser, timeout=10).until(EC.presence_of_element_located((By.XPATH, "//span[@class='section-header-label']")))
#     elements = browser.find_elements_by_xpath("//li[@class='com-coupon-line avb-row market-avb large ']")
#     elements[index].click()
#     WebDriverWait(browser, timeout=10).until(EC.presence_of_element_located((By.XPATH, "//div[@class='ui-expandable ui-expandable-selected com-expandable-header-anchor has-deeplink']")))
#     return browser.page_source

def parse(html):
  soup = BeautifulSoup(html, "html.parser")
  pll = soup.findAll("div", "mod yui3-widget yui3-module yui3-minimarketview")
  for p in pll:
    player = p.find("span", text = re.compile(".*Total Points $"), attrs =  {"class": "title"})
    if player:
      printSoup(player)
      printOther(p)

def printOther(p):
  pp = p.findAll("li", attrs={'class': "runner-item"})
  for bla in pp:
    printOdd(bla)

def printOdd(bla):
  spans = bla.findAll("span")
  points = spans[2].find(text=True)
  odd = spans[1].find(text=True)
  print(points)
  print(odd)

def printSoup(soup):
  for a in soup:
    print(a)
    print()

if __name__ == '__main__':
  main()